System information (for reproducibility):

In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: macOS (arm64-apple-darwin21.5.0)
  CPU: 12 × Apple M2 Max
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, apple-m1)
  Threads: 8 on 8 virtual cores
Environment:
  JULIA_NUM_THREADS = 8
  JULIA_EDITOR = code


Load packages:

In [2]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/github.com/ucla-biostat-257/2023spring/slides/10-trisys`


Status `~/Documents/github.com/ucla-biostat-257/2023spring/slides/10-trisys/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random


For the next couple of lectures, we consider computer algorithms for solving linear equations $\mathbf{A} \mathbf{x} = \mathbf{b}$, a ubiquitous task in statistics. 

Key idea: turning original problem into an **easy** one, e.g., triangular system.

## Lower triangular system

To solve $\mathbf{A} \mathbf{x} = \mathbf{b}$, where $\mathbf{A} \in \mathbb{R}^{n \times n}$ is **lower triangular**

$$
\begin{pmatrix}
    a_{11} & 0 & \cdots & 0 \\
    a_{21} & a_{22} & \cdots & 0 \\
    \vdots & \vdots & \ddots & \vdots \\
    a_{n1} & a_{n2} & \cdots & a_{nn}
\end{pmatrix}
\begin{pmatrix}
x_1 \\ x_2 \\ \vdots \\ x_n
\end{pmatrix} = \begin{pmatrix}
b_1 \\ b_2 \\ \vdots \\ b_n
\end{pmatrix}.
$$

* **Forward substitution**: 
$$
\begin{eqnarray*}
    x_1 &=& b_1 / a_{11} \\
    x_2 &=& (b_2 - a_{21} x_1) / a_{22} \\
    x_3 &=& (b_3 - a_{31} x_1 - a_{32} x_2) / a_{33} \\
    &\vdots& \\
    x_n &=& (b_n - a_{n1} x_1 - a_{n2} x_2 - \cdots - a_{n,n-1} x_{n-1}) / a_{nn}.
\end{eqnarray*}
$$

* $1 + 3 + 5 + \cdots + (2n-1) = n^2$ flops. 

* $\mathbf{A}$ can be accessed by row ($ij$ loop) or column ($ji$ loop).

## Upper triangular system

To solve $\mathbf{A} \mathbf{x} = \mathbf{b}$, where $\mathbf{A} \in \mathbb{R}^{n \times n}$ is upper triangular  
$$
\begin{pmatrix}
    a_{11} & \cdots & a_{1,n-1} & a_{1n} \\
    \vdots & \ddots & \vdots & \vdots \\
    0 & \cdots & a_{n-1,n-1} & a_{n-1,n} \\
    0 & 0 & 0 & a_{nn}
\end{pmatrix}
\begin{pmatrix}
x_1 \\ \vdots \\ x_{n-1} \\ x_n
\end{pmatrix} = \begin{pmatrix}
b_1 \\ \vdots \\ b_{n-1} \\ b_n
\end{pmatrix}.
$$

* **Back substitution** 
$$
\begin{eqnarray*}
    x_n &=& b_n / a_{nn} \\
    x_{n-1} &=& (b_{n-1} - a_{n-1,n} x_n) / a_{n-1,n-1} \\
    x_{n-2} &=& (b_{n-2} - a_{n-2,n-1} x_{n-1} - a_{n-2,n} x_n) / a_{n-2,n-2} \\
    &\vdots& \\
    x_1 &=& (b_1 - a_{12} x_2 - a_{13} x_3 - \cdots - a_{1,n} x_{n}) / a_{11}.
\end{eqnarray*}
$$

* $n^2$ flops.

* $\mathbf{A}$ can be accessed by row ($ij$ loop) or column ($ji$ loop).

## Implementation

* BLAS level 2 function: [?trsv](http://www.netlib.org/lapack/explore-html/d6/d96/dtrsv_8f.html) (triangular solve with one right hand side).

* BLAS level 3 function: [?trsm](http://www.netlib.org/lapack/explore-html/de/da7/dtrsm_8f.html) (matrix triangular solve, i.e., multiple right hand sides).

* Julia  
    - The left divide `\` operator in Julia is used for solving linear equations or least squares problem.  
    - If `A` is a triangular matrix, the command `A \ b` uses forward or backward substitution  
    - Or we can call the BLAS wrapper functions directly: [trsv!](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.BLAS.trsv!), [trsv](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.BLAS.trsv), [trsm!](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.BLAS.trsm!), [trsm](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.BLAS.trsm)

In [3]:
using LinearAlgebra, Random

Random.seed!(123) # seed
n = 5
A = randn(n, n)
b = randn(n)
# a random matrix
A

5×5 Matrix{Float64}:
  0.808288   0.229819    1.21928    -0.890077   2.00811
 -1.12207   -0.421769    0.292914    0.854242   0.76503
 -1.10464   -1.35559    -0.0311481   0.341782   0.180254
 -0.416993   0.0694591   0.315833   -0.31887    2.02891
  0.287588  -0.117323   -2.16238    -0.337454  -1.08822

In [4]:
Al = LowerTriangular(A) # does not create extra matrix

5×5 LowerTriangular{Float64, Matrix{Float64}}:
  0.808288    ⋅           ⋅           ⋅          ⋅ 
 -1.12207   -0.421769     ⋅           ⋅          ⋅ 
 -1.10464   -1.35559    -0.0311481    ⋅          ⋅ 
 -0.416993   0.0694591   0.315833   -0.31887     ⋅ 
  0.287588  -0.117323   -2.16238    -0.337454  -1.08822

In [5]:
dump(Al)

LowerTriangular{Float64, Matrix{Float64}}
  data: Array{Float64}((5, 5)) [0.8082879284649668 0.2298186980518676 … -0.8900766562698114 2.008112624852031; -1.1220725081141734 -0.4217686643996927 … 0.8542424475591821 0.7650300179102025; … ; -0.4169926351649334 0.0694591410918936 … -0.3188700715740966 2.0289114854525314; 0.28758798062385577 -0.11732280453081337 … -0.33745447783309457 -1.088218513936287]


In [6]:
Al.data

5×5 Matrix{Float64}:
  0.808288   0.229819    1.21928    -0.890077   2.00811
 -1.12207   -0.421769    0.292914    0.854242   0.76503
 -1.10464   -1.35559    -0.0311481   0.341782   0.180254
 -0.416993   0.0694591   0.315833   -0.31887    2.02891
  0.287588  -0.117323   -2.16238    -0.337454  -1.08822

In [7]:
# same data
pointer(Al.data), pointer(A)

(Ptr{Float64} @0x000000017e438640, Ptr{Float64} @0x000000017e438640)

In [8]:
Al \ b # dispatched to BLAS function for triangular solve

5-element Vector{Float64}:
    0.8706777634658246
   -2.656170478296127
   79.95736189623057
   74.31241954954413
 -181.43591336155936

In [9]:
# or use BLAS wrapper directly
BLAS.trsv('L', 'N', 'N', A, b)

5-element Vector{Float64}:
    0.8706777634658246
   -2.656170478296127
   79.95736189623057
   74.31241954954413
 -181.43591336155936

In [10]:
?BLAS.trsv

```
trsv(ul, tA, dA, A, b)
```

Return the solution to `A*x = b` or one of the other two variants determined by [`tA`](@ref stdlib-blas-trans) and [`ul`](@ref stdlib-blas-uplo). [`dA`](@ref stdlib-blas-diag) determines if the diagonal values are read or are assumed to be all ones.


* Some other BLAS functions for triangular systems such as triangular matrix-vector and triangular matrix-matrix multiplications: [trmv](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.BLAS.trmv), [trmv!](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.BLAS.trmv!), [trmm](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.BLAS.trmm), [trmm!](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.BLAS.trmm!)

## Some algebraic facts of triangular system (HW1)

* Eigenvalues of a triangular matrix $\mathbf{A}$ are diagonal entries $\lambda_i = a_{ii}$. 

* Determinant $\det(\mathbf{A}) = \prod_i a_{ii}$.

* The product of two upper (lower) triangular matrices is upper (lower) triangular.

* The inverse of an upper (lower) triangular matrix is upper (lower) triangular.

* The product of two unit upper (lower) triangular matrices is unit upper (lower) triangular.

* The inverse of a unit upper (lower) triangular matrix is unit upper (lower) triangular.

## Julia types for triangular matrices

[LowerTriangular](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.LowerTriangular), UnitLowerTriangular, 
[UpperTriangular](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.UpperTriangular), UnitUpperTriangular.  

In [11]:
A

5×5 Matrix{Float64}:
  0.808288   0.229819    1.21928    -0.890077   2.00811
 -1.12207   -0.421769    0.292914    0.854242   0.76503
 -1.10464   -1.35559    -0.0311481   0.341782   0.180254
 -0.416993   0.0694591   0.315833   -0.31887    2.02891
  0.287588  -0.117323   -2.16238    -0.337454  -1.08822

In [12]:
LowerTriangular(A)

5×5 LowerTriangular{Float64, Matrix{Float64}}:
  0.808288    ⋅           ⋅           ⋅          ⋅ 
 -1.12207   -0.421769     ⋅           ⋅          ⋅ 
 -1.10464   -1.35559    -0.0311481    ⋅          ⋅ 
 -0.416993   0.0694591   0.315833   -0.31887     ⋅ 
  0.287588  -0.117323   -2.16238    -0.337454  -1.08822

In [13]:
LinearAlgebra.UnitLowerTriangular(A)

5×5 UnitLowerTriangular{Float64, Matrix{Float64}}:
  1.0         ⋅           ⋅          ⋅         ⋅ 
 -1.12207    1.0          ⋅          ⋅         ⋅ 
 -1.10464   -1.35559     1.0         ⋅         ⋅ 
 -0.416993   0.0694591   0.315833   1.0        ⋅ 
  0.287588  -0.117323   -2.16238   -0.337454  1.0

In [14]:
using BenchmarkTools, LinearAlgebra, Random

Random.seed!(123) # seed
A = randn(1000, 1000);

In [15]:
# if we don't tell Julia it's triangular: O(n^3) complexity
# tril(A) returns a full triangular matrix, same as Matlab
@benchmark eigvals($(tril(A)))

BenchmarkTools.Trial: 210 samples with 1 evaluation.
 Range (min … max):  22.901 ms …  25.828 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     23.874 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   23.875 ms ± 545.500 μs  ┊ GC (mean ± σ):  0.67% ± 1.48%

    ▁           ▂█▇     ▃ ▇▅▃ ▃                                 
  ▅▄█▄▄▃▆▄▆▄▄▅▅▆████▄▃▅▆█▇███▆██▇▁▆▄▃▅▆▁▅▃▃▃▄▁▁▃▁▅▃▁▃▄▄▃▃▁▁▃▁▃ ▄
  22.9 ms         Histogram: frequency by time         25.4 ms <

 Memory estimate: 7.92 MiB, allocs estimate: 13.

In [16]:
# if we tell Julia it's triangular: O(n) complexity
@benchmark eigvals($(LowerTriangular(A)))

BenchmarkTools.Trial: 10000 samples with 198 evaluations.
 Range (min … max):  432.449 ns … 12.835 μs  ┊ GC (min … max):  0.00% … 93.18%
 Time  (median):     733.689 ns              ┊ GC (median):     0.00%
 Time  (mean ± σ):   938.933 ns ±  1.135 μs  ┊ GC (mean ± σ):  22.67% ± 16.36%

  ▆▂█▇▅▄                                                       ▂
  ███████▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▄▅▇█████▇ █
  432 ns        Histogram: log(frequency) by time      6.98 μs <

 Memory estimate: 7.94 KiB, allocs estimate: 1.

In [17]:
# if we don't tell Julia it's triangular: O(n^3) complexity
# tril(A) returns a full triangular matrix, same as Matlab
@benchmark det($(tril(A)))

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  211.250 μs … 300.458 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     212.334 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   214.115 μs ±   4.918 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▇▇▆▅▄▂ ▄▄▄▄▃▃▂▁   ▁▁▂▁▁                                      ▂
  ███████▇███████████████████▇██▇███▇▆▆▅▆▆▆▆▅▅▅▄▃▄▄▃▃▃▅▃▄▅▅▄▄▄▄ █
  211 μs        Histogram: log(frequency) by time        236 μs <

 Memory estimate: 7.94 KiB, allocs estimate: 1.

In [18]:
# if we tell Julia it's triangular: O(n) complexity
@benchmark det($(LowerTriangular(A)))

BenchmarkTools.Trial: 10000 samples with 157 evaluations.
 Range (min … max):  669.586 ns … 13.243 μs  ┊ GC (min … max):  0.00% … 89.80%
 Time  (median):     984.611 ns              ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.231 μs ±  1.429 μs  ┊ GC (mean ± σ):  19.50% ± 14.75%

  ▄██▆▂                                                        ▂
  █████▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▇████ █
  670 ns        Histogram: log(frequency) by time      9.67 μs <

 Memory estimate: 7.94 KiB, allocs estimate: 1.

In [19]:
@benchmark det($(LowerTriangular(A)))

BenchmarkTools.Trial: 10000 samples with 157 evaluations.
 Range (min … max):  669.586 ns … 15.095 μs  ┊ GC (min … max):  0.00% … 90.30%
 Time  (median):     971.070 ns              ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.215 μs ±  1.418 μs  ┊ GC (mean ± σ):  19.62% ± 14.76%

  ▅██▅▂                                                     ▁▁ ▂
  █████▅▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▅▇███ █
  670 ns        Histogram: log(frequency) by time      9.56 μs <

 Memory estimate: 7.94 KiB, allocs estimate: 1.